# **HYPERPARAMETER OPTIMIZATION**

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X, y = load_iris(return_X_y=True)
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
import random

def random_chrom():
    return [
        random.randint(10, 200),   # número de árboles
        random.randint(2, 10),     # profundidad máxima
        random.randint(2, 10)      # numero de muestras
    ]

def fitness(chrom):
    n_estimators, max_depth, min_samples_split = chrom
    clf = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    clf.fit(Xtr, ytr)
    return clf.score(Xval, yval)

def tournament_selection(pop, fitnesses, k=3):
    selected = []
    for _ in range(len(pop)):
        aspirants = random.sample(list(range(len(pop))), k)
        winner = max(aspirants, key=lambda i: fitnesses[i])
        selected.append(pop[winner])
    return selected

def one_point_crossover(p1, p2):
    point = random.randint(1, len(p1)-1)
    return p1[:point]+p2[point:], p2[:point]+p1[point:]

def mutate(chrom, pm=0.3):
    if random.random() < pm:
        chrom[0] = random.randint(10,200)
    if random.random() < pm:
        chrom[1] = random.randint(2,10)
    if random.random() < pm:
        chrom[2] = random.randint(2,10)
    return chrom

In [3]:
POP_SIZE = 6
GENERATIONS = 5

pop = [random_chrom() for _ in range(POP_SIZE)]
print("Población inicial:", pop)

fitnesses = [fitness(ch) for ch in pop]

for gen in range(GENERATIONS):
    print(f"\n=== Generación {gen} ===")
    for i, (chrom, fit) in enumerate(zip(pop, fitnesses)):
        print(f" Individuo {i}: {chrom} → Fitness={fit:.4f}")

    parents = tournament_selection(pop, fitnesses, k=3)

    next_pop = []
    for i in range(0, POP_SIZE, 2):
        c1, c2 = one_point_crossover(parents[i], parents[(i+1)%POP_SIZE])
        next_pop += [c1, c2]

    next_pop = [mutate(c) for c in next_pop]
    print("   Después de mutación:", next_pop)

    fitnesses = [fitness(ch) for ch in next_pop]
    pop = next_pop

best_idx = max(range(len(fitnesses)), key=lambda i: fitnesses[i])
print("\n=== Mejor solución encontrada ===")
print("Mejores hiperparámetros:", pop[best_idx])
print("Fitness:", fitnesses[best_idx])

Población inicial: [[187, 6, 2], [76, 10, 6], [113, 4, 6], [129, 9, 8], [32, 8, 2], [80, 3, 10]]

=== Generación 0 ===
 Individuo 0: [187, 6, 2] → Fitness=1.0000
 Individuo 1: [76, 10, 6] → Fitness=1.0000
 Individuo 2: [113, 4, 6] → Fitness=1.0000
 Individuo 3: [129, 9, 8] → Fitness=1.0000
 Individuo 4: [32, 8, 2] → Fitness=1.0000
 Individuo 5: [80, 3, 10] → Fitness=1.0000
   Después de mutación: [[113, 6, 2], [187, 4, 6], [80, 9, 8], [129, 4, 9], [80, 10, 6], [76, 3, 10]]

=== Generación 1 ===
 Individuo 0: [113, 6, 2] → Fitness=1.0000
 Individuo 1: [187, 4, 6] → Fitness=1.0000
 Individuo 2: [80, 9, 8] → Fitness=1.0000
 Individuo 3: [129, 4, 9] → Fitness=1.0000
 Individuo 4: [80, 10, 6] → Fitness=1.0000
 Individuo 5: [76, 3, 10] → Fitness=1.0000
   Después de mutación: [[129, 10, 6], [80, 10, 9], [73, 9, 6], [75, 7, 8], [80, 7, 6], [83, 4, 6]]

=== Generación 2 ===
 Individuo 0: [129, 10, 6] → Fitness=1.0000
 Individuo 1: [80, 10, 9] → Fitness=1.0000
 Individuo 2: [73, 9, 6] → Fitness